### 04 - kNN classification
De dataset met alle keypoints en aanvullende informatie zal gebruikt worden om een kNN classificatiemodel te trainen.

In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import pandas as pd
import pickle

#### Stap 1. Train- en testdata genereren

In [18]:
# Keuze voor model maken

train_df = pd.read_csv('keypoints_train_df_YOLOv8m.csv', index_col=0)
test_df = pd.read_csv('keypoints_test_df_YOLOv8m.csv', index_col=0)

#train_df = pd.read_csv('keypoints_train_df_YOLOv8n.csv', index_col=0)
#test_df = pd.read_csv('keypoints_test_df_YOLOv8n.csv', index_col=0)

In [14]:
train_df.head()

,path,pose,pose_code,confidence,x00,y00,x01,y01,x02,y02,...,y14_norm,y15_norm,y16_norm,A,B,C,D,E,F,G
0,C:\Users\joost.vanstreels\PycharmProjects\YOLO...,crossed,0,0.914894,0.519160,0.314611,0.534587,0.305057,0.500624,0.305733,...,1.253703,1.967561,1.957170,-0.277956,0.288862,0.784445,0.856234,0.717169,0.804296,0.004347
1,C:\Users\joost.vanstreels\PycharmProjects\YOLO...,crossed,0,0.909827,0.509721,0.315518,0.524141,0.306418,0.489993,0.307169,...,1.268211,2.011203,1.988737,0.234085,0.257445,0.700592,0.724724,0.742652,0.801409,0.008705
2,C:\Users\joost.vanstreels\PycharmProjects\YOLO...,crossed,0,0.906767,0.503642,0.196350,0.515195,0.174214,0.488498,0.178452,...,1.237465,1.396445,1.381261,-0.056527,0.083777,0.570295,0.518169,0.876922,0.938895,0.004897
3,C:\Users\joost.vanstreels\PycharmProjects\YOLO...,crossed,0,0.912517,0.497531,0.165511,0.515113,0.152043,0.480580,0.154027,...,1.271745,1.989151,1.987445,-0.476262,0.476776,0.754658,0.754125,0.914735,0.917351,0.011187
4,C:\Users\joost.vanstreels\PycharmProjects\YOLO...,crossed,0,0.901814,0.542198,0.162231,0.560624,0.147076,0.521831,0.149408,...,1.259467,1.940902,1.949706,-0.395487,0.395488,0.753799,0.698847,0.920348,0.917714,0.002302


In [15]:
features = ['x07_norm', 'x08_norm', 'x09_norm', 'x10_norm', 'x11_norm', 'x12_norm',
            'y07_norm', 'y08_norm', 'y09_norm', 'y10_norm', 'y11_norm', 'y12_norm',
            'v00',
            'A', 'B', 'C', 'D', 'E', 'F', 'G']

X_train = train_df[features]
X_train

,x07_norm,x08_norm,x09_norm,x10_norm,x11_norm,x12_norm,y07_norm,y08_norm,y09_norm,y10_norm,y11_norm,y12_norm,v00,A,B,C,D,E,F,G
0,0.639115,-0.563602,-0.050594,0.227362,0.381690,-0.402223,-0.089824,-0.062856,-0.158934,-0.080310,0.495653,0.500000,0.980632,-0.277956,0.288862,0.784445,0.856234,0.717169,0.804296,0.004347
1,0.825788,-0.713876,0.258069,0.023984,0.336785,-0.410158,-0.041827,-0.027412,-0.196156,-0.089003,0.500000,0.491295,0.985178,0.234085,0.257445,0.700592,0.724724,0.742652,0.801409,0.008705
2,0.196583,-0.199889,-0.023439,0.033088,0.123211,-0.150940,-0.000445,-0.011576,-0.051118,0.010714,0.500000,0.495103,0.984101,-0.056527,0.083777,0.570295,0.518169,0.876922,0.938895,0.004897
3,0.439619,-0.382674,-0.258366,0.217897,0.290757,-0.297021,-0.003023,0.001724,-0.028848,-0.050966,0.488813,0.500000,0.990279,-0.476262,0.476776,0.754658,0.754125,0.914735,0.917351,0.011187
4,0.431464,-0.381640,-0.251316,0.144171,0.277918,-0.299598,0.032868,0.008405,-0.039074,-0.039867,0.497698,0.500000,0.987471,-0.395487,0.395488,0.753799,0.698847,0.920348,0.917714,0.002302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,0.582055,-0.468545,0.944042,-0.343267,0.531026,0.052732,-0.108130,0.220464,0.282820,0.636526,0.393955,0.500000,0.988299,1.287309,1.335018,0.427707,0.418873,1.637076,1.346452,0.106045
244,0.485350,-0.559456,0.543343,-0.591194,0.252850,-0.284802,0.041233,0.038932,0.497691,0.520584,0.500000,0.491301,0.992305,1.134537,1.134768,0.290503,0.307788,1.486253,1.575971,0.008699
245,0.564844,-0.549005,0.863498,-0.316569,0.521971,0.006466,-0.068861,0.194554,0.330750,0.632521,0.401332,0.500000,0.989026,1.180067,1.218041,0.348744,0.349161,1.601692,1.473962,0.098668
246,0.424467,-0.426122,0.520251,-0.440308,0.318547,-0.113303,0.005752,0.076118,0.424738,0.553667,0.468660,0.500000,0.988724,0.960559,0.969173,0.206430,0.331380,1.394131,1.574170,0.031340


In [16]:
y_train = train_df['pose_code']
y_train

0      0
1      0
2      0
3      0
4      0
      ..
243    5
244    5
245    5
246    5
247    5
Name: pose_code, Length: 248, dtype: int64

In [19]:
X_test.head()

,x00,y00,x01,y01,x02,y02,x03,y03,x04,y04,...,A,B,C,D,E,F,G,H,I,J
0,0.480748,0.071525,0.510249,0.057365,0.456121,0.057642,0.550774,0.082610,0.421157,0.081954,...,-0.080448,0.134302,0.163137,0.208280,0.017981,0.135700,0.113652,0.260541,0.283223,0.000634
1,0.505474,0.135134,0.528415,0.119128,0.484709,0.119614,0.567540,0.126171,0.453870,0.129224,...,-0.107289,0.106815,0.129682,0.129198,0.123472,0.130368,0.104537,0.189634,0.214230,0.001235
2,0.494083,0.177466,0.519363,0.160807,0.468841,0.162995,0.561741,0.168188,0.436529,0.171919,...,-0.005530,0.167503,0.158724,0.021241,0.138608,0.072251,0.088643,0.257825,0.248250,0.006818
3,0.482926,0.127723,0.505874,0.116070,0.467202,0.114994,0.551718,0.132172,0.450347,0.128645,...,-0.112145,0.097619,0.108904,0.111268,0.145803,0.113742,0.107329,0.194888,0.197926,0.003376
4,0.490134,0.242953,0.502809,0.218792,0.476563,0.216520,0.515500,0.231187,0.451240,0.225084,...,0.091885,0.157386,0.164110,0.017719,0.021130,0.211350,0.203848,0.288076,0.297258,0.001681


In [20]:
X_test = test_df[features]
X_test

,x07_norm,x08_norm,x09_norm,x10_norm,x11_norm,x12_norm,y07_norm,y08_norm,y09_norm,y10_norm,y11_norm,y12_norm,v00,A,B,C,D,E,F,G
0,0.074643,-0.633680,-0.404620,-0.113973,0.347867,-0.178936,0.070238,0.037921,0.007444,0.089391,0.497708,0.500000,0.987407,-0.290647,0.301978,0.898107,0.415716,1.047857,1.037230,0.002292
1,0.369311,-0.603798,-0.373119,0.079228,0.171600,-0.441349,0.005187,0.005649,-0.049651,0.054049,0.500000,0.494792,0.991631,-0.452347,0.464081,0.773844,0.682096,0.874134,0.908632,0.005208
2,0.398222,-0.533842,0.206791,0.229145,0.292661,-0.331187,0.083014,0.024285,0.180361,0.141654,0.472439,0.500000,0.990969,-0.022354,0.044699,0.304439,0.665120,1.061052,1.027632,0.027561
3,0.381529,-0.597109,-0.288024,0.222637,0.218644,-0.441290,0.081317,0.022896,-0.017931,-0.004100,0.500000,0.484629,0.989792,-0.510661,0.510848,0.724545,0.824411,0.907726,0.956330,0.015371
4,0.268835,-0.254051,0.089249,-0.158811,0.137086,-0.101766,-0.047920,0.002361,-0.075111,-0.050323,0.495462,0.500000,0.987095,0.248060,0.249296,0.572575,0.553271,0.779865,0.824709,0.004538
5,0.682860,-0.596466,-0.101095,0.294614,0.322126,-0.442435,-0.082768,-0.062644,-0.160131,-0.077193,0.497823,0.500000,0.985989,-0.395709,0.404307,0.782317,0.936159,0.725293,0.844534,0.002177
6,0.514707,-0.439550,0.081768,0.091611,0.357924,-0.258158,-0.011599,0.073268,0.018507,0.148086,0.484326,0.500000,0.987456,-0.009843,0.129952,0.541526,0.496167,0.989005,1.114714,0.015674
7,0.443683,-0.595750,-0.357241,0.059794,0.195968,-0.354637,0.021247,0.085627,0.064892,0.007966,0.472853,0.500000,0.967766,-0.417035,0.420903,0.687366,0.643312,0.716489,0.616081,0.027147
8,0.502502,-0.592266,-0.200168,-0.105783,0.287193,-0.402547,0.080847,0.119559,0.053327,0.125762,0.494723,0.500000,0.989206,-0.094385,0.118977,0.657535,0.477623,0.937023,0.992105,0.005277
9,0.487388,-0.475532,0.055601,0.162031,0.370766,-0.280687,0.074795,0.176835,0.047730,0.186486,0.477142,0.500000,0.989073,-0.106430,0.174874,0.532657,0.542485,0.924875,1.075523,0.022858


In [21]:
y_test = test_df['pose_code']
y_test

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    1
12    1
13    1
14    1
15    1
16    1
17    1
18    1
19    1
20    1
21    1
22    2
23    2
24    2
25    2
26    2
27    2
28    2
29    3
30    3
31    3
32    3
33    3
34    3
35    3
36    3
37    4
38    4
39    4
40    4
41    4
42    4
43    4
44    4
45    5
46    5
47    5
48    5
49    5
50    5
Name: pose_code, dtype: int64

#### Stap 2. Model trainen

TO DO: GridSearchCV, spelen met features, kijken welk model het beter doet
TO DO: o.b.v. eigen logica

In [42]:
knn = KNeighborsClassifier()
knn.set_params(n_neighbors = 5, weights = 'uniform')
knn.fit(X_train, y_train)

KNeighborsClassifier()

#### Stap 3. Model evalueren


In [43]:
y_train_pred = knn.predict(X_train)
accuracy_score(y_train, y_train_pred)

0.9516129032258065

In [44]:
confusion_matrix(y_train, y_train_pred)

array([[33,  0,  0,  0,  0,  0],
       [ 0, 71,  0,  0,  0,  0],
       [ 0,  0, 41,  0,  1,  0],
       [ 0,  1,  0, 35,  0,  0],
       [ 3,  3,  0,  0, 16,  0],
       [ 0,  4,  0,  0,  0, 40]], dtype=int64)

In [45]:
y_test_pred = knn.predict(X_test)
accuracy_score(y_test, y_test_pred)

0.9411764705882353

In [46]:
knn.predict_proba(X_test)

array([[1. , 0. , 0. , 0. , 0. , 0. ],
       [1. , 0. , 0. , 0. , 0. , 0. ],
       [1. , 0. , 0. , 0. , 0. , 0. ],
       [1. , 0. , 0. , 0. , 0. , 0. ],
       [1. , 0. , 0. , 0. , 0. , 0. ],
       [1. , 0. , 0. , 0. , 0. , 0. ],
       [1. , 0. , 0. , 0. , 0. , 0. ],
       [1. , 0. , 0. , 0. , 0. , 0. ],
       [1. , 0. , 0. , 0. , 0. , 0. ],
       [1. , 0. , 0. , 0. , 0. , 0. ],
       [1. , 0. , 0. , 0. , 0. , 0. ],
       [1. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 1. , 0. , 0. , 0. , 0. ],
       [0. , 1. , 0. , 0. , 0. , 0. ],
       [0. , 1. , 0. , 0. , 0. , 0. ],
       [0. , 0.6, 0. , 0. , 0. , 0.4],
       [0. , 1. , 0. , 0. , 0. , 0. ],
       [0. , 0.6, 0. , 0.2, 0.2, 0. ],
       [0. , 1. , 0. , 0. , 0. , 0. ],
       [0. , 1. , 0. , 0. , 0. , 0. ],
       [0. , 1. , 0. , 0. , 0. , 0. ],
       [0. , 1. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 1. , 0. , 0. , 0. ],
       [0. , 0. , 1. , 0. , 0. , 0. ],
       [0. , 0. , 1. , 0. , 0. , 0. ],
       [0. , 0. , 1. , 0.

In [33]:
confusion_matrix(y_test, y_test_pred)

array([[11,  0,  0,  0,  0,  0],
       [ 1,  9,  0,  1,  0,  0],
       [ 0,  0,  7,  0,  0,  0],
       [ 0,  0,  0,  8,  0,  0],
       [ 0,  0,  0,  0,  7,  1],
       [ 0,  0,  0,  0,  0,  6]], dtype=int64)

#### Stap 4. Model en voorspellingen exporteren

In [34]:
# Prediction en proba toevoegen aan DataFrame

train_df['prediction'] = y_train_pred
train_df['proba'] = knn.predict_proba(X_train).max(axis=1)
train_df.head()

,path,pose,pose_code,confidence,x00,y00,x01,y01,x02,y02,...,y16_norm,A,B,C,D,E,F,G,prediction,proba
0,C:\Users\joost.vanstreels\PycharmProjects\YOLO...,crossed,0,0.914894,0.519160,0.314611,0.534587,0.305057,0.500624,0.305733,...,1.957170,-0.277956,0.288862,0.784445,0.856234,0.717169,0.804296,0.004347,0,1.0
1,C:\Users\joost.vanstreels\PycharmProjects\YOLO...,crossed,0,0.909827,0.509721,0.315518,0.524141,0.306418,0.489993,0.307169,...,1.988737,0.234085,0.257445,0.700592,0.724724,0.742652,0.801409,0.008705,0,1.0
2,C:\Users\joost.vanstreels\PycharmProjects\YOLO...,crossed,0,0.906767,0.503642,0.196350,0.515195,0.174214,0.488498,0.178452,...,1.381261,-0.056527,0.083777,0.570295,0.518169,0.876922,0.938895,0.004897,0,1.0
3,C:\Users\joost.vanstreels\PycharmProjects\YOLO...,crossed,0,0.912517,0.497531,0.165511,0.515113,0.152043,0.480580,0.154027,...,1.987445,-0.476262,0.476776,0.754658,0.754125,0.914735,0.917351,0.011187,0,1.0
4,C:\Users\joost.vanstreels\PycharmProjects\YOLO...,crossed,0,0.901814,0.542198,0.162231,0.560624,0.147076,0.521831,0.149408,...,1.949706,-0.395487,0.395488,0.753799,0.698847,0.920348,0.917714,0.002302,0,1.0


In [35]:
# DataFrame exporteren
# Keuze maken voor model

train_df.to_csv('train_df_YOLOv8m.csv')
#train_df.to_csv('train_df_YOLOv8n.csv')

In [36]:
# Pickle maken
# Keuze maken voor model

pickle.dump(knn, open('knn_YOLOv8m.pkl', 'wb'))
#pickle.dump(knn, open('knn_YOLOv8n.pkl', 'wb'))